In [1]:
#import libraries
import pandas as pd
import seaborn as sns
import numpy as np
import sklearn
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Flatten
from tensorflow.keras.models import Sequential
from math import sqrt
import math
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
import pickle

### Training the model 

In [2]:
### built LSTM model
keras.backend.clear_session()
np.random.seed(7)
tf.random.set_seed(7)

In [3]:
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

#### cluster 1

In [7]:
def data():
    X_train = np.load('ISO_NE/X_train1_isone.npy')
    X_test = np.load('ISO_NE/X_val1_isone.npy')
    Y_train = np.load('ISO_NE/Y_train1_isone.npy')
    Y_test = np.load('ISO_NE/Y_val1_isone.npy')
    
    return X_train, Y_train, X_test, Y_test

In [8]:
#LSTM for cluster1
def model(X_train, Y_train, X_test, Y_test):
    '''
    Model providing function:
    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
        
        
    '''
    from keras.regularizers import l2
    len_data    = len(X_train)
    batch_size  = 25
    learning_rate_fn = 0.0001
    model_mlp_sch = Sequential()
    inner_layers = {{choice(['two', 'three','four'])}}
    first_layer_neurons = {{choice([100, 200 ,300, 400, 600, 800, 1000])}}
    second_layer_neurons = {{choice([100, 200 ,300, 400, 600])}}
    third_layer_neurons = {{choice([100, 200 ,300])}}
    forth_layer_neurons = {{choice([50, 100 ,150])}}
    activations = {{choice(['relu', 'sigmoid','linear','tanh'])}}
    content = {{choice(['kernel_initializer','no'])}}
    epochs = {{choice([70,100,130,170,200])}}

    
    if  inner_layers == 'two':
        if content == 'kernel_initializer':
            model_mlp_sch.add(LSTM(first_layer_neurons, return_sequences=True,activation=activations,kernel_initializer='lecun_normal',  input_shape=( len(X_train[0]), len(X_train[0][0]) )))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(second_layer_neurons, activation=activations,kernel_initializer='lecun_normal'))
            model_mlp_sch.add(Dropout(0.3))
        
        elif content == 'no':
            model_mlp_sch.add(LSTM(first_layer_neurons, return_sequences=True,activation=activations,  input_shape=( len(X_train[0]), len(X_train[0][0]) )))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(second_layer_neurons, activation=activations ))
            model_mlp_sch.add(Dropout(0.3))
    elif inner_layers == 'three':
        if content == 'kernel_initializer':
            model_mlp_sch.add(LSTM(first_layer_neurons,return_sequences=True, activation=activations,kernel_initializer='lecun_normal',  input_shape=( len(X_train[0]), len(X_train[0][0]) )))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(second_layer_neurons,return_sequences=True, activation=activations,kernel_initializer='lecun_normal'))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(third_layer_neurons, activation=activations,kernel_initializer='lecun_normal'))
            model_mlp_sch.add(Dropout(0.3))
        
        elif content == 'no':
            model_mlp_sch.add(LSTM(first_layer_neurons,return_sequences=True, activation=activations,  input_shape=( len(X_train[0]), len(X_train[0][0]) )))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(second_layer_neurons,return_sequences=True, activation=activations ))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(third_layer_neurons, activation=activations ))
            model_mlp_sch.add(Dropout(0.3))
    else:
        if content == 'kernel_initializer':
            model_mlp_sch.add(LSTM(first_layer_neurons, return_sequences=True, activation=activations,kernel_initializer='lecun_normal',  input_shape=( len(X_train[0]), len(X_train[0][0]) )))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(second_layer_neurons, return_sequences=True, activation=activations,kernel_initializer='lecun_normal'))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(third_layer_neurons, return_sequences=True, activation=activations,kernel_initializer='lecun_normal'))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(forth_layer_neurons, activation=activations,kernel_initializer='lecun_normal'))
            model_mlp_sch.add(Dropout(0.3))
       
        elif content == 'no':
            model_mlp_sch.add(LSTM(first_layer_neurons, return_sequences=True, activation=activations,  input_shape=( len(X_train[0]), len(X_train[0][0]) )))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(second_layer_neurons, return_sequences=True, activation=activations ))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(third_layer_neurons, return_sequences=True, activation=activations ))
            model_mlp_sch.add(Dropout(0.3))
            model_mlp_sch.add(LSTM(forth_layer_neurons, activation=activations ))
            model_mlp_sch.add(Dropout(0.3))
    
    model_mlp_sch.add(Dense(len(Y_train[0]),activation='linear'))
    model_mlp_sch.compile(optimizer= tf.keras.optimizers.Adam( learning_rate= learning_rate_fn ), loss='mape')
    history_mlp = model_mlp_sch.fit(X_train, Y_train,validation_data = (X_test , Y_test) , epochs=epochs, verbose=0)
    
    Test_predictions = model_mlp_sch.predict(X_test)
    
    return {'loss': mape(Y_test,Test_predictions), 'status': STATUS_OK, 'model': model_mlp_sch}

In [9]:
best_run, best_model= optim.minimize(model=model, data = data, max_evals=100, algo=tpe.suggest,
                                          notebook_name='hyperas_Final_3models', # This is important!
                                          trials=Trials(),
                                            functions=[mape],
                                     eval_space=True,   # <-- this is the line that puts real values into 'best_run'
                                     #return_space=True  # <-- this allows you to save the space for later evaluations
                                        )


>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import seaborn as sns
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import sklearn
except:
    pass

try:
    from sklearn.metrics import mean_squared_error, r2_score
except:
    pass

try:
    from datetime import datetime
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, Flatten
except:
    pass

try:
    from tensorflow.keras.models import Sequential
except:
    pass

try:
    from math import sqrt
except:
    pass

try:
    import math
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import pickle
except:
    pass

try:
    from keras.regularizer

KeyboardInterrupt: 

In [8]:
print('cluster1')
print(best_run)

cluster1
{'activations': 'sigmoid', 'content': 'no', 'epochs': 20, 'first_layer_neurons': 400, 'forth_layer_neurons': 150, 'inner_layers': 'two', 'second_layer_neurons': 200, 'third_layer_neurons': 300}


In [13]:
#save best parameters
best_parameters_cluster1_isone = open("best_parameters_cluster1_isone.pkl", "wb")
pickle.dump(best_run, best_parameters_cluster1_isone)
best_parameters_cluster1_isone.close()

In [14]:
""" 
a_file = open("best_parameters_cluster1_isone.pkl", "rb")
output = pickle.load(a_file)
print(output)
"""

{'activations': 'sigmoid', 'content': 'no', 'epochs': 20, 'first_layer_neurons': 400, 'forth_layer_neurons': 150, 'inner_layers': 'two', 'second_layer_neurons': 200, 'third_layer_neurons': 300}


In [ ]:
best_model.save('hyp_cluster1_isone.h5')

#### cluster 2

In [ ]:
def data():
    X_train = np.load('X_train2_isone.npy')
    X_test = np.load('X_val2_isone.npy')
    Y_train = np.load('Y_train2_isone.npy')
    Y_test = np.load('Y_val2_isone.npy')
    
    return X_train, Y_train, X_test, Y_test

In [ ]:
best_run, best_model= optim.minimize(model=model,
                                       data = data,
                                       max_evals=100,
                                          algo=tpe.suggest,
                                          notebook_name='hyperas_Final_3models', # This is important!
                                          trials=Trials(),
                                            functions=[mape],
                                     eval_space=True,   # <-- this is the line that puts real values into 'best_run'
                                     #return_space=True  # <-- this allows you to save the space for later evaluations
                                        )


In [ ]:
print('cluster2')
print(best_run)

In [ ]:
best_parameters_cluster2_isone = open("best_parameters_cluster2_isone.pkl", "wb")
pickle.dump(best_run, best_parameters_cluster2_isone)
best_parameters_cluster2_isone.close()

In [ ]:
best_model.save('hyp_cluster2_isone.h5')

#### Cluster 3

In [ ]:
def data():
    X_train = np.load('X_train3_isone.npy')
    X_test = np.load('X_val3_isone.npy')
    Y_train = np.load('Y_train3_isone.npy')
    Y_test = np.load('Y_val3_isone.npy')
    
    return X_train, Y_train, X_test, Y_test

In [ ]:
best_run, best_model= optim.minimize(model=model,
                                       data = data,
                                       max_evals=100,
                                          algo=tpe.suggest,
                                          notebook_name='hyperas_Final_3models', # This is important!
                                          trials=Trials(),
                                            functions=[mape],
                                     eval_space=True,   # <-- this is the line that puts real values into 'best_run'
                                     #return_space=True  # <-- this allows you to save the space for later evaluations
                                        )


In [ ]:
print('cluster3')
print(best_run)

In [ ]:
#save parameters
best_parameters_cluster1_isone = open("best_parameters_cluster1_isone.pkl", "wb")
pickle.dump(best_run, best_parameters_cluster1_isone)
best_parameters_cluster1_isone.close()

In [ ]:
best_model.save('hyp_cluster3_isone.h5')